# SageMaker V3 Train-to-Inference E2E Example

This notebook demonstrates the complete end-to-end workflow from training a model to deploying it for inference using SageMaker V3.

In [ ]:
# Import required libraries
import json
import uuid

from sagemaker.serve.model_builder import ModelBuilder
from sagemaker.train.model_trainer import ModelTrainer
from sagemaker.utils.jumpstart.configs import JumpStartConfig
from sagemaker.core.resources import EndpointConfig

## Step 1: Configure Training Job

Set up a JumpStart model for training. We'll use a smaller model for faster training.

In [ ]:
# Configuration for training
MODEL_ID = "huggingface-llm-falcon-7b-bf16"  # You can use smaller models for faster training
TRAINING_JOB_PREFIX = "e2e-training-job"
MODEL_NAME_PREFIX = "e2e-trained-model"
ENDPOINT_NAME_PREFIX = "e2e-trained-endpoint"

# Generate unique identifiers
unique_id = str(uuid.uuid4())[:8]
training_job_name = f"{TRAINING_JOB_PREFIX}-{unique_id}"
model_name = f"{MODEL_NAME_PREFIX}-{unique_id}"
endpoint_name = f"{ENDPOINT_NAME_PREFIX}-{unique_id}"

print(f"Training job name: {training_job_name}")
print(f"Model name: {model_name}")
print(f"Endpoint name: {endpoint_name}")

## Step 2: Create ModelTrainer

Set up the ModelTrainer with JumpStart configuration for training.

In [ ]:
# Create ModelTrainer from JumpStart config
jumpstart_config = JumpStartConfig(model_id=MODEL_ID)
model_trainer = ModelTrainer.from_jumpstart_config(jumpstart_config=jumpstart_config)

print("ModelTrainer created successfully from JumpStart config!")
print(f"Ready to train model: {MODEL_ID}")

## Step 3: Start Training Job

Launch the training job. This will take significant time depending on the model size and data.

In [ ]:
# Start training job
print(f"Starting training job: {training_job_name}")
print("Note: This may take 30+ minutes depending on the model and dataset size.")

model_trainer.train(training_job_name=training_job_name)
print(f"Training completed successfully: {training_job_name}")

## Step 4: Build ModelBuilder from Training Artifacts

Create a ModelBuilder from the trained model artifacts.

In [ ]:
# Build ModelBuilder from trained model
print("Building ModelBuilder from training artifacts...")
model_builder = model_trainer.build()

print("ModelBuilder successfully created from training artifacts!")
print("Ready to deploy the trained model.")

## Step 5: Build the Trained Model

Build the model artifacts for deployment.

In [ ]:
# Build the trained model
core_model = model_builder.build(model_name=model_name)
print(f"Trained Model Successfully Created: {core_model.model_name}")

## Step 6: Deploy the Trained Model

Deploy the trained model to a SageMaker endpoint.

In [ ]:
# Deploy the trained model
core_endpoint = model_builder.deploy(endpoint_name=endpoint_name)
print(f"Trained Model Endpoint Successfully Created: {core_endpoint.endpoint_name}")

## Step 7: Test the Trained Model

Test the deployed trained model with sample inputs.

In [ ]:
# Test the trained model
test_data = {"inputs": "What did you learn during training?", "parameters": {"max_new_tokens": 50}}

result = core_endpoint.invoke(
    body=json.dumps(test_data),
    content_type="application/json"
)

# Decode and display the result
prediction = json.loads(result.body.read().decode('utf-8'))
print(f"Trained Model Response: {prediction}")

In [ ]:
# Test with different prompts to see training effects
test_prompts = [
    "Explain machine learning in simple terms.",
    "What is the capital of France?",
    "Write a short poem about AI."
]

for i, prompt in enumerate(test_prompts, 1):
    test_data = {"inputs": prompt, "parameters": {"max_new_tokens": 30}}
    
    result = core_endpoint.invoke(
        body=json.dumps(test_data),
        content_type="application/json"
    )
    
    prediction = json.loads(result.body.read().decode('utf-8'))
    print(f"Test {i} - '{prompt}': {prediction}")
    print("-" * 50)

## Step 8: Clean Up Resources

Clean up all created resources including the trained model and endpoint.

In [ ]:
# Clean up resources
core_endpoint_config = EndpointConfig.get(endpoint_config_name=core_endpoint.endpoint_name)

# Delete in the correct order
core_model.delete()
core_endpoint.delete()
core_endpoint_config.delete()

print("All trained model resources successfully deleted!")
print(f"Note: Training job {training_job_name} artifacts remain in S3 for reference.")

## Summary

This notebook demonstrated the complete E2E workflow:
1. Configuring a ModelTrainer with JumpStart
2. Running a training job on SageMaker
3. Building a ModelBuilder from training artifacts
4. Deploying the trained model to an endpoint
5. Testing the trained model's performance
6. Proper cleanup of inference resources

## Key Benefits of E2E Training:
- **Seamless workflow**: Train → Build → Deploy in one pipeline
- **Artifact management**: Automatic handling of training outputs
- **Custom training**: Full control over training process
- **Production ready**: Trained models ready for immediate deployment

The V3 ModelBuilder makes it easy to go from training to production inference with minimal code!